# Context Encoders : Feature learning by Inpainting

### Project By:
1. Naman Jain (2020201080)
2. Ayush Khasgiwala (2020201088)
3. Archit Gupta (2020201075)

## Importing Dependencies

In [ ]:
from __future__ import print_function, division
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import shutil
import os
import glob
import cv2

## Data Preprocessing

In [ ]:
!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d puneet6060/intel-image-classification
!ls -l ~/.kaggle

-rw-r--r-- 1 root root 66 Apr 27 23:53 kaggle.json
 98% 338M/346M [00:02<00:00, 171MB/s]
100% 346M/346M [00:02<00:00, 138MB/s]
total 4
-rw------- 1 root root 66 Apr 27 23:53 kaggle.json


In [ ]:
from zipfile import ZipFile
print("uncompressing zip file")

filename = "intel-image-classification.zip"

with ZipFile(filename, 'r') as zip:
    zip.extractall()

print("done")

uncompressing zip file
done


In [ ]:
def create_directory(d_name):
    directory = d_name
    mode = 0o666
    path = "/content/"+directory
    os.mkdir(path, mode)

create_directory("train_images")
create_directory("test_images")
create_directory("out_validation")
create_directory("out_test")

In [ ]:
def write_img(dname,img,img_name):
    path="/content/"+dname+"/"+img_name
    wr=cv2.imwrite(path, img)

def process_data(img_path, i, dname):
    img = cv2.imread(img_path)
    dim = (150, 150)  
    
    # resize image  
    resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)  

    path = str (i) + '.jpg'
    write_img(dname, resized, path)

In [ ]:
train_folder = "/content/seg_train/seg_train/*"
train_dname = "train_images"
i = 0
for folder in glob.glob(train_folder):
    if folder in ["/content/seg_train/seg_train/buildings", "/content/seg_train/seg_train/street"]:
        continue

    for img in glob.glob(folder + "/*"):
        process_data(img, i, train_dname)
        i+=1
    print(str (i) + ". " + folder + "   =====> completed") 

# -----------------------------------------------------------------------------#

test_folder = "/content/seg_test/seg_test/*"
test_dname = "test_images"
i = 0
for folder in glob.glob(test_folder):
    if folder in ["/content/seg_test/seg_test/buildings", "/content/seg_test/seg_test/street"]:
        continue
    for img in glob.glob(folder + "/*"):
        process_data(img, i, test_dname)
        i+=1
    print(str (i) + ". " + folder + "   =====> completed") 


2274. /content/seg_train/seg_train/sea   =====> completed
4678. /content/seg_train/seg_train/glacier   =====> completed
6949. /content/seg_train/seg_train/forest   =====> completed
9461. /content/seg_train/seg_train/mountain   =====> completed
510. /content/seg_test/seg_test/sea   =====> completed
1063. /content/seg_test/seg_test/glacier   =====> completed
1537. /content/seg_test/seg_test/forest   =====> completed
2062. /content/seg_test/seg_test/mountain   =====> completed


In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

train_folder = "/content/train_images"
test_folder = "/content/test_images"

X_train = load_images_from_folder(train_folder)
X_test = load_images_from_folder(test_folder)

X_train = np.array(X_train)
X_test = np.array(X_test)
print(X_train.shape)
print(X_test.shape)

(9461, 150, 150, 3)
(2062, 150, 150, 3)


## Normalization

In [ ]:
X_train = X_train / 127.5 - 1.
X_test = X_test / 127.5 - 1.

## Creating Random Masks for patch generation

In [ ]:
def get_coordinates():
    x1 = np.random.randint(0, 150 - 40, imgs.shape[0])
    y1 = np.random.randint(0, 150 - 40, imgs.shape[0])
    x2 = x1 + 40
    y2 = y1 + 40

    return x1, y1, x2, y2

In [ ]:
def mask_randomly(imgs):
    x1, y1, x2, y2 = get_coordinates()

    masked_imgs = np.empty_like(imgs)
    missing_parts = np.empty((imgs.shape[0], 40, 40, 3))

    for i, img in enumerate(imgs):
        masked_img = img.copy()
        _x1 = x1[i]
        _y1 = y1[i]
        _x2 = x2[i]
        _y2 = y2[i]
  
        missing_parts[i] = masked_img[_y1:_y2, _x1:_x2, :].copy()
        masked_img[_y1:_y2, _x1:_x2, :] = 0
        masked_imgs[i] = masked_img

    return masked_imgs, missing_parts, (y1, y2, x1, x2)

## Generator Architecture

In [ ]:
def build_generator():
    model = Sequential()

    # ------------------------------- ENCODER ---------------------------------#

    model.add(Conv2D(32, kernel_size=3, strides = 2, input_shape = (150,150,3), padding="same"))
    model.add(LeakyReLU()
    model.add(BatchNormalization()

    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU()
    model.add(BatchNormalization()

    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU()
    model.add(BatchNormalization()

    model.add(Conv2D(512, kernel_size=1, strides=2, padding="same"))
    model.add(LeakyReLU()
    model.add(Dropout(0.5))

    # ------------------------------- DECODER ---------------------------------#
    
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization()

    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization()

    model.add(Conv2D(3, kernel_size=3, padding="same"))
    model.add(Activation('tanh'))

    model.summary()

    masked_img = Input(shape=(150,150,3))
    gen_missing = model(masked_img)

    return Model(masked_img, gen_missing)

## Discriminator Architecture

In [ ]:
def build_discriminator():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape = (40, 40, 3), padding="same"))
    model.add(LeakyReLU()
    model.add(BatchNormalization()

    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU()
    model.add(BatchNormalization()

    model.add(Conv2D(256, kernel_size=3, padding="same"))
    model.add(LeakyReLU()
    model.add(BatchNormalization()

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()

    img = Input(shape=(40, 40, 3))
    validity = model(img)

    return Model(img, validity)

In [ ]:
generator = build_generator()
masked_img = Input((150,150,3))
gen_missing = generator(masked_img)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 75, 75, 32)        896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 75, 75, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 75, 75, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 38, 38, 64)        18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 38, 38, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 38, 38, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 19, 128)       7

In [ ]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
valid = discriminator(gen_missing)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 20, 20, 32)        896       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 20, 20, 32)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 20, 20, 32)        128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 10, 10, 128)       36992     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 10, 10, 128)       0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 10, 10, 128)       512       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 10, 10, 256)      

## Combined compilation

In [ ]:
combined = Model(masked_img , [gen_missing, valid])
combined.compile(loss = ['mse', 'binary_crossentropy'], loss_weights = [0.999, 0.001], optimizer = 'adam')

## Save Images

In [ ]:
def save_images(epoch, imgs, folder):
    rows = 1
    columns = 3

    if folder=="train":
        idx = np.random.randint(0,2)
    else:
        idx = 0

    masked_imgs, missing_parts, (y1, y2, x1, x2) = mask_randomly(imgs)
    gen_missing = generator.predict(masked_imgs)

    imgs = 0.5 * imgs + 0.5
    masked_imgs = 0.5 * masked_imgs + 0.5
    gen_missing = 0.5 * gen_missing + 0.5

    fig = plt.figure(figsize=(15, 6))

    fig.add_subplot(rows, columns, 1)
    plt.imshow(imgs[idx])
    plt.axis('off')
    plt.title("Original Image")

    fig.add_subplot(rows, columns, 2)
    plt.imshow(masked_imgs[idx])
    plt.axis('off')
    plt.title("Masked Image")

    fig.add_subplot(rows, columns, 3)
    filled_in = imgs[idx].copy()
    filled_in[y1[idx]:y2[idx], x1[idx]:x2[idx], :] = gen_missing[idx]
    plt.imshow(filled_in)
    plt.axis('off')
    plt.title("Generated Image")

    if folder=="train":
        fig.savefig("out_validation/%d.png" % epoch)
    else:
        fig.savefig("out_test/%d.png" % epoch)
    plt.close()


## Training Loop

In [ ]:
batch_size = 32
sample_interval = 100

valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(10001):

    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    masked_imgs, missing_parts, _ = mask_randomly(imgs)
    gen_missing = generator.predict(masked_imgs)

    d_loss_real = discriminator.train_on_batch(missing_parts, valid)
    d_loss_fake = discriminator.train_on_batch(gen_missing, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) #adversial
    g_loss = combined.train_on_batch(masked_imgs, [missing_parts, valid])

    print ("Epoch %d :\t D loss: %f \t G loss: %f \t\t L2 loss: %f" % (epoch, d_loss[0], g_loss[0], g_loss[1]))

    if epoch % sample_interval == 0:
        idx = np.random.randint(0, X_train.shape[0], 2)
        imgs = X_train[idx]
        save_images(epoch, imgs, "train")


Streaming output truncated to the last 5000 lines.
Epoch 5001 :	 D loss: 0.000000 	 G loss: 0.226010 		 L2 loss: 0.226235
Epoch 5002 :	 D loss: 0.000000 	 G loss: 0.212433 		 L2 loss: 0.212645
Epoch 5003 :	 D loss: 0.000000 	 G loss: 0.222700 		 L2 loss: 0.222923
Epoch 5004 :	 D loss: 0.000000 	 G loss: 0.202893 		 L2 loss: 0.203095
Epoch 5005 :	 D loss: 0.000000 	 G loss: 0.195791 		 L2 loss: 0.195987
Epoch 5006 :	 D loss: 0.000000 	 G loss: 0.228248 		 L2 loss: 0.228431
Epoch 5007 :	 D loss: 0.000000 	 G loss: 0.229590 		 L2 loss: 0.229818
Epoch 5008 :	 D loss: 0.000000 	 G loss: 0.218969 		 L2 loss: 0.219089
Epoch 5009 :	 D loss: 0.000000 	 G loss: 0.226444 		 L2 loss: 0.226670
Epoch 5010 :	 D loss: 0.000000 	 G loss: 0.184473 		 L2 loss: 0.184657
Epoch 5011 :	 D loss: 0.000000 	 G loss: 0.214114 		 L2 loss: 0.214326
Epoch 5012 :	 D loss: 0.000000 	 G loss: 0.205306 		 L2 loss: 0.205511
Epoch 5013 :	 D loss: 0.000000 	 G loss: 0.150232 		 L2 loss: 0.150319
Epoch 5014 :	 D loss: 0.00

## Saving Model

In [ ]:
generator.save_weights('generator.h5')
discriminator.save_weights('discriminator.h5')
combined.save_weights('combined.h5')

## Reloading saved weights

In [ ]:
generator.load_weights('generator.h5')
discriminator.load_weights('discriminator.h5')
combined.load_weights('combined.h5')

## Testing Loop

In [ ]:
def run_test_loop(X_test, generator, discriminator, combined):
    batch_size = 1

    print("STATS")
    print("-----\n")

    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(101):
        
        idx = np.random.randint(epoch, epoch+1, batch_size)

        imgs = X_test[idx]
        masked_imgs, missing_parts, _ = mask_randomly(imgs)
        gen_missing = generator.predict(masked_imgs)

        d_loss_real = discriminator.train_on_batch(missing_parts, valid)
        d_loss_fake = discriminator.train_on_batch(gen_missing, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 
        g_loss = combined.train_on_batch(masked_imgs, [missing_parts, valid])

        print ("# %d :\t D loss: %f \t G loss: %f \t\t L2 loss: %f" % (epoch, d_loss[0], g_loss[0], g_loss[1]))

        imgs = X_test[idx]
        save_images(epoch, imgs, "test")

In [ ]:
run_test_loop(X_test, generator, discriminator, combined)

STATS
-----

# 0 :	 D loss: 0.000000 	 G loss: 0.517685 		 L2 loss: 0.518203
# 1 :	 D loss: 0.000000 	 G loss: 0.491744 		 L2 loss: 0.492236
# 2 :	 D loss: 0.000000 	 G loss: 0.414417 		 L2 loss: 0.414832
# 3 :	 D loss: 0.000000 	 G loss: 0.412867 		 L2 loss: 0.413280
# 4 :	 D loss: 0.000000 	 G loss: 0.505542 		 L2 loss: 0.506048
# 5 :	 D loss: 0.000000 	 G loss: 0.577459 		 L2 loss: 0.578037
# 6 :	 D loss: 0.000000 	 G loss: 0.329713 		 L2 loss: 0.330043
# 7 :	 D loss: 0.000000 	 G loss: 0.247225 		 L2 loss: 0.243700
# 8 :	 D loss: 80.369080 	 G loss: 0.479559 		 L2 loss: 0.480039
# 9 :	 D loss: 0.006964 	 G loss: 0.154552 		 L2 loss: 0.154707
# 10 :	 D loss: 35.939724 	 G loss: 0.329072 		 L2 loss: 0.329401
# 11 :	 D loss: 0.000000 	 G loss: 0.255573 		 L2 loss: 0.255829
# 12 :	 D loss: 53.493351 	 G loss: 0.884988 		 L2 loss: 0.885873
# 13 :	 D loss: 0.000000 	 G loss: 0.675104 		 L2 loss: 0.644462
# 14 :	 D loss: 0.000000 	 G loss: 0.255490 		 L2 loss: 0.190738
# 15 :	 D loss: 60.

In [ ]:
shutil.make_archive("validation_output", 'zip', "/content/out_validation")
shutil.make_archive("test_output", 'zip', "/content/out_test")

'/content/test_output.zip'